In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 假設你的檔案在 /MyDrive/datasets/celeba.zip
!unzip "/content/drive/MyDrive/img_align_celeba.zip" -d "/content/datasets/celeba"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/MyDrive/img_align_celeba.zip
checkdir:  cannot create extraction directory: /content/datasets/celeba
           No such file or directory


In [ ]:
# ==== 0. 基本設定（掛載雲端、解壓、資料集路徑） ====
import os
from google.colab import drive

# 掛載雲端
drive.mount('/content/drive')

ZIP_PATH = "/content/drive/MyDrive/img_align_celeba.zip"  # 你的 zip 檔路徑
EXTRACT_DIR = "/content/datasets/celeba/img_align_celeba" # 解壓後圖片所在資料夾
SAMPLES_DIR = "/content/drive/MyDrive/celeba_gan_samples" # 生成結果輸出資料夾
os.makedirs(SAMPLES_DIR, exist_ok=True)

# 解壓（若尚未解壓才會執行）
if not os.path.exists(EXTRACT_DIR):
    os.makedirs(os.path.dirname(EXTRACT_DIR), exist_ok=True)
    !unzip -q "{ZIP_PATH}" -d "/content/datasets/celeba"
else:
    print("🚀 已找到解壓後資料夾，略過解壓。")

# 檢查前幾個檔案
!ls -l "/content/datasets/celeba" | head -n 20
!ls -l "{EXTRACT_DIR}" | head -n 10

# ==== 1. 匯入套件 ====
import glob
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader

# ==== 2. 自訂平面影像資料集（無類別資料夾也可讀） ====
class FlatImageFolder(Dataset):
    def __init__(self, root, transform=None, exts=("jpg","jpeg","png","bmp","webp")):
        self.root = root
        self.transform = transform
        files = []
        for ext in exts:
            files.extend(glob.glob(os.path.join(root, f"**/*.{ext}"), recursive=True))
            files.extend(glob.glob(os.path.join(root, f"*.{ext}")))
        self.files = sorted(list(set(files)))
        if len(self.files) == 0:
            raise RuntimeError(f"No images found under: {root}")
        print(f"🖼️  找到 {len(self.files)} 張影像（於 {root}）")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fp = self.files[idx]
        img = Image.open(fp).convert("RGB")
        if self.transform:
            img = self.transform(img)
        # 這裡不需要 label，回傳 dummy 0
        return img, 0

# ==== 3. 參數與資料載入 ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️  Device:", device)

image_size = 64
batch_size = 128
nz = 100     # latent dim
ngf = 64     # G base channels
ndf = 64     # D base channels
epochs = 5   # 示範

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

dataset = FlatImageFolder(EXTRACT_DIR, transform=transform)
dataloader = DataLoader(
    dataset, batch_size=batch_size, shuffle=True,
    num_workers=2, pin_memory=(device.type=="cuda")
)

# ==== 4. DCGAN 模型（D 無 Sigmoid，配 BCEWithLogitsLoss） ====
class Generator(nn.Module):
    def __init__(self, nz=100, ngf=64, nc=3):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False),  # 1x1 -> 4x4
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1, bias=False),  # 4x4 -> 8x8
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),  # 8x8 -> 16x16
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),    # 16x16 -> 32x32
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),       # 32x32 -> 64x64
            nn.Tanh()
        )
    def forward(self, z):
        return self.main(z)

class Discriminator(nn.Module):
    def __init__(self, ndf=64, nc=3):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),      # 64->32
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),   # 32->16
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False), # 16->8
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*4, ndf*8, 4, 2, 1, bias=False), # 8->4
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf*8, 1, 4, 1, 0, bias=False)      # 4->1
            # 無 Sigmoid
        )
    def forward(self, x):
        return self.main(x).view(-1)

netG = Generator(nz, ngf).to(device)
netD = Discriminator(ndf).to(device)

# 權重初始化（DCGAN 建議 N(0, 0.02)）
def weights_init(m):
    cname = m.__class__.__name__
    if cname.find('Conv') != -1 or cname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    if cname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.zeros_(m.bias.data)

netG.apply(weights_init)
netD.apply(weights_init)

criterion = nn.BCEWithLogitsLoss()
optimizerD = optim.Adam(netD.parameters(), lr=2e-4, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=2e-4, betas=(0.5, 0.999))

# 固定噪聲，追蹤訓練進度
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# ==== 5. 訓練 ====
netG.train(); netD.train()

for epoch in range(1, epochs+1):
    for i, (real, _) in enumerate(dataloader):
        real = real.to(device)
        bsz = real.size(0)

        # 標籤（可採 label smoothing：真實=0.9）
        real_labels = torch.full((bsz,), 0.9, device=device)
        fake_labels = torch.zeros(bsz, device=device)

        # ---- 更新 D ----
        optimizerD.zero_grad()
        pred_real = netD(real)
        lossD_real = criterion(pred_real, real_labels)

        noise = torch.randn(bsz, nz, 1, 1, device=device)
        fake = netG(noise).detach()
        pred_fake = netD(fake)
        lossD_fake = criterion(pred_fake, fake_labels)

        lossD = lossD_real + lossD_fake
        lossD.backward()
        optimizerD.step()

        # ---- 更新 G ----
        optimizerG.zero_grad()
        noise = torch.randn(bsz, nz, 1, 1, device=device)
        gen = netG(noise)
        pred = netD(gen)
        lossG = criterion(pred, torch.ones(bsz, device=device))
        lossG.backward()
        optimizerG.step()

    print(f"[Epoch {epoch}/{epochs}]  LossD: {lossD.item():.4f}  LossG: {lossG.item():.4f}")

    # 以固定噪聲匯出觀察圖
    netG.eval()
    with torch.no_grad():
        samples = netG(fixed_noise).cpu()
        save_path = os.path.join(SAMPLES_DIR, f"epoch_{epoch:03d}.png")
        save_image(samples, save_path, normalize=True, value_range=(-1, 1), nrow=8)
        print("💾 Saved:", save_path)
    netG.train()

    # （可選）儲存 ckpt
    torch.save({
        "G": netG.state_dict(),
        "D": netD.state_dict(),
        "optG": optimizerG.state_dict(),
        "optD": optimizerD.state_dict(),
        "epoch": epoch
    }, os.path.join(SAMPLES_DIR, f"ckpt_{epoch:03d}.pt"))

print("✅ 訓練完成！檔案已輸出到：", SAMPLES_DIR)

# ==== 6. 產生一批新的人臉樣本（收尾） ====
netG.eval()
with torch.no_grad():
    z = torch.randn(64, nz, 1, 1, device=device)
    gen_samples = netG(z).cpu()
    save_image(gen_samples, os.path.join(SAMPLES_DIR, "final_samples.png"),
               normalize=True, value_range=(-1, 1), nrow=8)
print("📷 另存一批最終樣本：final_samples.png")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total 5772
drwxr-xr-x 2 root root 5906432 Sep 28  2015 img_align_celeba
total 1737936
-rw-r--r-- 1 root root 11440 Sep 28  2015 000001.jpg
-rw-r--r-- 1 root root  7448 Sep 28  2015 000002.jpg
-rw-r--r-- 1 root root  4253 Sep 28  2015 000003.jpg
-rw-r--r-- 1 root root 10747 Sep 28  2015 000004.jpg
-rw-r--r-- 1 root root  6351 Sep 28  2015 000005.jpg
-rw-r--r-- 1 root root  8073 Sep 28  2015 000006.jpg
-rw-r--r-- 1 root root  8203 Sep 28  2015 000007.jpg
-rw-r--r-- 1 root root  7725 Sep 28  2015 000008.jpg
-rw-r--r-- 1 root root  8641 Sep 28  2015 000009.jpg
🖥️  Device: cuda
🖼️  找到 202599 張影像（於 /content/datasets/celeba/img_align_celeba）
[Epoch 1/5]  LossD: 0.6953  LossG: 4.3539
💾 Saved: /content/drive/MyDrive/celeba_gan_samples/epoch_001.png
[Epoch 2/5]  LossD: 1.2970  LossG: 1.0415
💾 Saved: /content/drive/MyDrive/celeba_gan_samples/epoch_002.png
[Epoch 3/5]  L